In [4]:
# ===============================
# Connect Four Simulation Notebook
# ===============================

import numpy as np
from itertools import combinations

# Import functions from SRC
from src.board import create_board, make_move, check_win
from src.player import random_player, heuristic_player, intelligent_player
from src.simulation import simulate_game

# --- Parameters ---
BASE_SIM = 100  # default number of simulations per pairing

# Define players
players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

# Optional: automatic simulation count per pairing (between 100-500)
def determine_sim_count(balance_estimate):
    """Return simulation count based on estimated balance"""
    return int(np.clip(BASE_SIM * (1 + balance_estimate * 2), 100, 500))

# Estimated balance between players (0 = unbalanced, 1 = very balanced)
balance_estimates = {
    ("4 Years","7 Years"): 0.1,
    ("4 Years","9 Years"): 0.3,
    ("4 Years","11 Years"): 0.05,
    ("7 Years","9 Years"): 0.5,
    ("7 Years","11 Years"): 0.7,
    ("9 Years","11 Years"): 0.8
}

# --- Run all simulations with both start players ---
for p1_name, p2_name in combinations(players.keys(), 2):

    sim_count = determine_sim_count(balance_estimates.get((p1_name, p2_name), 0.3))

    for start_order in [(p1_name, p2_name), (p2_name, p1_name)]:
        starter, second = start_order

        outcomes = {starter:0, second:0, "Draw":0}

        for _ in range(sim_count):
            winner = simulate_game(players[starter], players[second],
                                            moves_count_p1=None,
                                            moves_count_p2=None,
                                            wins_count_p1=None,
                                            wins_count_p2=None)
            if winner == 1:
                outcomes[starter] += 1
            elif winner == 2:
                outcomes[second] += 1
            else:
                outcomes["Draw"] += 1

        # Calculate probabilities
        prob_starter = outcomes[starter] / sim_count
        prob_second = outcomes[second] / sim_count
        prob_draw = outcomes["Draw"] / sim_count

        # Standard errors
        se_starter = np.sqrt(prob_starter * (1 - prob_starter) / sim_count)
        se_second = np.sqrt(prob_second * (1 - prob_second) / sim_count)
        se_draw = np.sqrt(prob_draw * (1 - prob_draw) / sim_count)

        # Betting odds
        quote_starter = 1 / prob_starter if prob_starter > 0 else float('inf')
        quote_second = 1 / prob_second if prob_second > 0 else float('inf')
        quote_draw = 1 / prob_draw if prob_draw > 0 else float('inf')

        # --- Print results ---
        print(f"\n{starter} vs {second} ({sim_count} games) - {starter} starts")
        print(f"{starter}: {prob_starter*100:.1f}% ± {se_starter*100:.1f}% → Quote ≈ {quote_starter:.2f}")
        print(f"{second}: {prob_second*100:.1f}% ± {se_second*100:.1f}% → Quote ≈ {quote_second:.2f}")
        print(f"Draw: {prob_draw*100:.1f}% ± {se_draw*100:.1f}% → Quote ≈ {quote_draw:.2f}")


ModuleNotFoundError: No module named 'src'